In [123]:
import scanpy as sc
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearColorMapper,ColorBar, LinearAxis, Range1d
from bokeh.models.transforms import CustomJSTransform
from bokeh.transform import factor_cmap, transform
import sys
sys.path.insert(0, '../beehive/')
import numpy as np
from beehive import config, util, expset
from bokeh.palettes import Category20
import polars as pl

In [ ]:
output_notebook()

In [3]:
dataset_id = 'h.man2m.1.sct'

In [ ]:
expset.obslist(dataset_id)

In [210]:
def get_data(dataset_id, gene1, gene2, obsfield):
    rv = pd.DataFrame(dict(
        x = expset.get_gene(dataset_id, gene1)[:,0],
        y = expset.get_gene(dataset_id, gene2)[:,0],
        obs = expset.get_meta(dataset_id, obsfield)[:,0],
        ))
    return rv
    
data = get_data(dataset_id, 'TREM2', 'APOE', 'injection')


In [ ]:
p = figure()
ages = list(data['obs'].unique())
source = ColumnDataSource(data)
p.scatter(x='x', y='y', source=source,  legend_field="obs",
          fill_alpha=0.2, size=3,
         color=factor_cmap('obs', 'Category10_3', ages))
show(p)

In [ ]:

datadir = util.get_datadir("h5ad")
dsid = "m.how1m.1"
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.var.prq",["field"]+["APOE"])
rv.to_pandas()

In [212]:

datadir = util.get_datadir("h5ad")
dsid = "h.s423a.1"
varsfields = expset.get_varfields(dsid)
categ = "plaque_bin__all"
len(pl.read_parquet(datadir / f"{dsid}.var.prq").columns)
rv1 = pl.read_parquet(datadir / f"{dsid}.var.prq",["plaque_bin__all__lfc"])
rv2 = pl.read_parquet(datadir / f"{dsid}.var.prq",[12])

#rv
# pl.concat([rv1,rv2],how="horizontal")

In [ ]:
vars = expset.get_varfields(dsid)
unique_vars = list(set([x.replace("__lfc","").replace("__padj","") for x in vars]))

In [11]:
categ = "ptau_bin__all"
dsid = "h.s423a.1"
data = expset.get_dedata_new(dsid,categ)

categ_x  = categ + "__lfc"
categ_y = categ + "__padj"
data[categ_y] = np.log10(data[categ_y]) * (-1)
plot = figure()
plot.scatter(x = categ_x, y = categ_y, source = ColumnDataSource(data))
show(plot)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
data = pd.DataFrame(dict(
        gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
        gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))
data

In [ ]:
unique_obs = expset.get_obsfields(dataset_id)

In [ ]:
expset.get_dataset(dsid)

In [91]:
datadir = util.get_datadir("h5ad")
dsid = "h.s423a.1"
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.var.prq")



In [206]:
p = figure()
dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id,'umap_1',raw=True)[:,0]
,
        gene2 = expset.get_meta(dataset_id,"umap_2",raw=True)[:,0]
,
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))
        
bins = hexbin(data["gene1"], data["gene2"], 0.1)
source_bin = ColumnDataSource(data=dict(r=[], q=[], counts=[]))
source_bin.data = dict(r=bins.r, q=bins.q, counts=bins.counts)

v_func_alpha  = """
var new_xs = new Array(xs.length)
for(var i = 0; i < xs.length; i++) {
    new_xs[i] = alpha_map[xs[i]]
}
return new_xs
"""
length = len(sorted(source_bin.data["counts"].unique().tolist()))
alpha_map = dict(zip(sorted(source_bin.data["counts"].unique().tolist()),[x/length for x in range(0,length)]))
numerical_alpha_transformer = CustomJSTransform(args={"alpha_map": alpha_map}, v_func=v_func_alpha)


#plot:
t = p.hex_tile(q="q",r="r",size=0.1,source=source_bin,
alpha = transform("counts",numerical_alpha_transformer),line_color = None, color = "black")
# p.scatter(x = "gene1",y="gene2", source = ColumnDataSource(data),size = 0.5,alpha=0.1)
show(p)


In [152]:
from bokeh.util.hex import cartesian_to_axial

###get Data####
# data = pd.DataFrame(dict(
#         gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
#         gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
#         obs = expset.get_meta(dataset_id, "cell.state")[:,0],
#         gene3 = expset.get_gene(dataset_id, "APOC1")[:,0],
#         ))

dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))


#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], 0.1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))
groups = df.groupby(["q","r"])
dicts = []
###assign axial coordinates to get average gene expression###
for key,val in groups.groups.items():
    q,r = key
    counts = len(val)
    avg_exp = data['gene3'].loc[groups.groups.get((q,r))].mean()
    dicts = dicts + [{"q" : q,"r" : r,"counts" : counts, "avg_exp" :avg_exp,"index_list":val}]
    

##use in source
final_result = pd.DataFrame(dicts)
source_bin = ColumnDataSource(final_result)



###mappers###
mapper = LinearColorMapper(
    palette='Magma256',
    low=final_result["avg_exp"].max(),
    high=final_result["avg_exp"].min())

v_func_alpha  = """
var new_xs = new Array(xs.length)
for(var i = 0; i < xs.length; i++) {
    new_xs[i] = alpha_map[xs[i]]
}
return new_xs
"""

###plot###
#make alpha optional => checkbox
#plot any continuous to any continuous
#subset for categorical fields => drop down (mutliselectfield)
#color of the third continious variable is based on the full data with the obs
#first layer is everything (with no colors one color only.) grey light
#second layer is the conitnuous variable colored + the possibility to subset
#have the option to either color or not color 
#widget for hex size
length = len(sorted(np.unique(source_bin.data["counts"]).tolist()))
alpha_map = dict(zip(sorted(np.unique(source_bin.data["counts"]).tolist()),[x/length for x in range(50,length)]))

numerical_alpha_transformer = CustomJSTransform(args={"alpha_map": alpha_map}, v_func=v_func_alpha)

# plot = figure()
# plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,
# alpha = transform("counts",numerical_alpha_transformer),line_color = None, 
# color  = {'field': 'avg_exp', 'transform': mapper})


plot = figure()
plot.hex_tile(q="q",r="r",size=0.1,source=source_bin, 
color  = "#D3D3D3")

plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,
alpha = transform("counts",numerical_alpha_transformer),line_color = None, 
color   = {'field': 'avg_exp', 'transform': mapper})

plot.xaxis.axis_label = "APOE"
plot.yaxis.axis_label = "TREM2"
color_bar1 = ColorBar(color_mapper=mapper, location=(0,0),major_label_text_font_size = "0px",major_tick_in = 2,title = "APOC1 average expression")
plot.add_layout(color_bar1,"right")
plot.extra_y_ranges = {"y2": Range1d(start = final_result["counts"].min(), end = final_result["counts"].max())}
extra_axis = LinearAxis(y_range_name = "y2",axis_label = "Counts by Transparency per Hex tile")
plot.add_layout(extra_axis, 'right')


show(plot)


In [149]:
final_result

,q,r,counts,avg_exp,index_list
0,-43,22,7,3.254143,"Int64Index([574, 2318, 6074, 9390, 21485, 2492..."
1,-43,23,76,3.192760,"Int64Index([ 93, 2280, 3264, 3508, 4056,..."
2,-43,24,15,2.974572,"Int64Index([ 912, 4476, 9967, 13571, 18521,..."
3,-43,25,1,0.693147,"Int64Index([2391], dtype='int64')"
4,-42,21,5,2.740728,"Int64Index([343, 6555, 17550, 28428, 54323], d..."
...,...,...,...,...,...
3108,42,-19,25,2.669467,"Int64Index([ 2253, 6858, 19522, 19852, 22166,..."
3109,42,-18,56,2.289125,"Int64Index([ 7656, 10784, 13020, 15045, 18550,..."
3110,42,-17,38,2.541593,"Int64Index([ 581, 4397, 8562, 8764, 12487,..."
3111,42,-16,12,2.840897,"Int64Index([23374, 26241, 28216, 28569, 30932,..."


In [188]:
categories = ["Homeostatic (HM)","Ribosomal response (RM)"]
data.loc[np.where(data["obs"].isin(categories))].reset_index(drop = True)

,gene1,gene2,gene3,obs
0,1.356828,0.264929,3.295837,Homeostatic (HM)
1,0.470477,0.478434,3.806662,Homeostatic (HM)
2,3.757786,2.785714,2.890372,Homeostatic (HM)
3,2.529936,1.216004,3.465736,Homeostatic (HM)
4,3.404747,-2.537671,2.708050,Ribosomal response (RM)
...,...,...,...,...
50965,4.705141,1.928640,2.484907,Homeostatic (HM)
50966,2.843638,1.596887,2.772589,Homeostatic (HM)
50967,3.424250,-0.355501,1.609438,Homeostatic (HM)
50968,0.947979,-3.486497,3.135494,Ribosomal response (RM)


In [184]:
type(data)

pandas.core.frame.DataFrame